In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

In [2]:
file_path = 'SurveyData.csv' 
data = pd.read_csv(file_path)
filtered_data = data[['Period', 'PayMonthly']]
cleaned_data = filtered_data.dropna()
# 为清晰起见，重命名列名
cleaned_data.columns = ['Subscription_Duration', 'Subscription_Price']
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.sans-serif'] = ['Hiragino Sans GB']

In [3]:
crosstab = pd.crosstab(cleaned_data['Subscription_Duration'], cleaned_data['Subscription_Price'])

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(crosstab, annot=True, fmt="d", cmap="YlGnBu")
plt.title("Subscription Duration vs Subscription Price")
plt.xlabel("Subscription Price (Category)")
plt.ylabel("Subscription Duration (Category)")
plt.show()

In [ ]:
chi2, p, dof, expected = chi2_contingency(crosstab)
print("Chi-Square Test Results:")
print(f"Chi-Square Statistic: {chi2:.2f}")
print(f"P-Value: {p:.4f}")
print(f"Degrees of Freedom: {dof}")

In [ ]:
if p < 0.05:
    print("结果显示：订阅时长与订阅价格之间存在显著的统计学关系 (P-Value < 0.05)。")
else:
    print("结果显示：订阅时长与订阅价格之间没有显著的统计学关系 (P-Value >= 0.05)。")
# 卡方检验的原假设 (H0)：订阅时长 (Subscription_Duration) 和订阅价格 (Subscription_Price) 是独立的。
# 备择假设 (H1)：订阅时长和订阅价格之间存在显著的统计学关系。

In [ ]:
desc_stats = cleaned_data.groupby('Subscription_Duration')['Subscription_Price'].describe()
print(desc_stats)
plt.figure(figsize=(8, 6))
sns.boxplot(x='Subscription_Duration', y='Subscription_Price', data=cleaned_data)
plt.title("Price Distribution by Subscription Duration")
plt.xlabel("Subscription Duration (Category)")
plt.ylabel("Subscription Price (Category)")
plt.show()

价格随订阅时长的变化
1个月和3个月订阅的价格偏低，集中在 6-15元/月，表明用户对短期订阅的价格较为敏感。
6个月和1年订阅的价格分布范围更广，中位数也更高，表明用户在长期订阅中愿意支付更高的价格。
价格的波动性
短期订阅（1个月和3个月）：价格波动较小，说明用户价格选择较为一致。
长期订阅（6个月和1年）：价格波动较大，表明用户偏好差异显著，可能受到个人需求或附加权益的影响。
特殊订阅时长
价格高度集中在 0-5元/月，可能是某种特定优惠订阅或试用计划，缺乏多样性。

高价格离群点（如 1个月的 26元以上/月 和 1年的 26元以上/月）：
表明部分用户可能对附加权益有强需求（如无广告、独占内容）。
也可能是特殊定价策略或服务（如家庭计划或超高音质服务）。
低价格离群点（如 6个月低于 6元/月）：
可能是特殊促销活动（如学生优惠计划）或数据录入异常。

In [8]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
import pandas as pd
file_path = 'SurveyData.csv'
data = pd.read_csv(file_path)
filtered_data = data[['Period', 'PayMonthly']]
# 填补 NaN 值
filtered_data.loc[:, 'Period'] = filtered_data['Period'].fillna(0).astype(int, errors='ignore')
filtered_data.loc[:, 'PayMonthly'] = filtered_data['PayMonthly'].fillna(0).astype(int, errors='ignore')
print("Period 唯一值：", filtered_data['Period'].unique())
print("PayMonthly 唯一值：", filtered_data['PayMonthly'].unique())
price_mapping = {
    1: 2.5,   # 0-5元
    2: 8,     # 6-10元
    3: 13,    # 11-15元
    4: 18,    # 16-20元
    5: 23,    # 21-25元
    6: 26.5,  # 26元以上
    0: 0      # 未订阅或无价格
}
duration_mapping = {
    1: 1,     # 1个月
    2: 3,     # 3个月
    3: 6,     # 6个月
    4: 12,    # 1年
    5: 15,    # 假设为15个月
    0: 0      # 未订阅或无时长
}
filtered_data.loc[:, 'Mapped_Price'] = filtered_data['PayMonthly'].map(price_mapping)
filtered_data.loc[:, 'Mapped_Duration'] = filtered_data['Period'].map(duration_mapping)
print("映射结果预览：")
print(filtered_data.head(20))
filtered_data = filtered_data[(filtered_data['Mapped_Price'] != 0) & (filtered_data['Mapped_Duration'] != 0)]
print("最终数据预览：")
print(filtered_data.head(20))


In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
plt.rcParams['font.sans-serif'] = ['Hiragino Sans GB']  # 设置字体为冬青黑
X = filtered_data['Mapped_Duration'].values.reshape(-1, 1)  
y = filtered_data['Mapped_Price'].values 
linear_model = LinearRegression()
linear_model.fit(X, y)
print(f"线性回归系数（Slope）：{linear_model.coef_[0]:.2f}")
print(f"线性回归截距（Intercept）：{linear_model.intercept_:.2f}")
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Mapped_Duration', y='Mapped_Price', data=filtered_data, color='purple', alpha=0.7)
plt.plot(filtered_data['Mapped_Duration'], linear_model.predict(X), color='red', label='Regression Line')
plt.title("线性回归：订阅价格与时长的关系")
plt.xlabel("订阅时长（月）")
plt.ylabel("订阅价格（元）")
plt.legend()
plt.show()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)  # degree=2 表示二次回归
X_poly = poly.fit_transform(X)
# 建立多项式回归模型
poly_model = LinearRegression()
poly_model.fit(X_poly, y)
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Mapped_Duration', y='Mapped_Price', data=filtered_data, color='purple', alpha=0.7)
plt.plot(filtered_data['Mapped_Duration'], poly_model.predict(X_poly), color='green', label='Polynomial Regression Curve')
plt.title("多项式回归：订阅价格与时长的关系")
plt.xlabel("订阅时长（月）")
plt.ylabel("订阅价格（元）")
plt.legend()
plt.show()


使用线性回归和多参数回归均为得出较为有效的结论，我们认为每月付费与订阅时长之间并无显著线性或拟线性联系

In [ ]:
# 使用分位数方法去除异常值
Q1_price = filtered_data['Mapped_Price'].quantile(0.25)
Q3_price = filtered_data['Mapped_Price'].quantile(0.75)
IQR_price = Q3_price - Q1_price
Q1_duration = filtered_data['Mapped_Duration'].quantile(0.25)
Q3_duration = filtered_data['Mapped_Duration'].quantile(0.75)
IQR_duration = Q3_duration - Q1_duration
filtered_data_cleaned = filtered_data[
    (filtered_data['Mapped_Price'] >= (Q1_price - 1.5 * IQR_price)) &
    (filtered_data['Mapped_Price'] <= (Q3_price + 1.5 * IQR_price)) &
    (filtered_data['Mapped_Duration'] >= (Q1_duration - 1.5 * IQR_duration)) &
    (filtered_data['Mapped_Duration'] <= (Q3_duration + 1.5 * IQR_duration))
]
print("清洗后的数据预览：")
print(filtered_data_cleaned.head())
# 再次绘制散点图
sns.scatterplot(
    x='Mapped_Duration', 
    y='Mapped_Price', 
    data=filtered_data_cleaned, 
    color='purple', 
    alpha=0.7
)
plt.title("清洗后的数据：订阅价格与时长的关系")
plt.xlabel("订阅时长（月）")
plt.ylabel("订阅价格（元）")
plt.show()

筛选异常数据后，我们发现每月的金额与订阅时长之间也无明显联系。在折合考虑后，我们选择直接以箱线图的结论作为结论。